# Algoritmo de Seleção de Endereços - Projeto Uber


## Objetivo
O objetivo deste algoritmo é gerar um conjunto de pares de endereços na cidade de São Paulo para o treinamento de um modelo de precificação de corridas de Uber.

## Metodologia
O algoritmo terá natureza estocástica, dando preferência para endereços de chegada em distritos com maior densidade populacional e renda média por habitante (fatores relevantes para o uso do aplicativo Uber).

Para os endereços de saída, o algoritmo usará uma função de decaimento exponencial $P(d)$ (descrita abaixo) para gerar uma distância máxima para o endereço do destino, onde escolherá o endereço de destino uniformemente dentro desse intervalo.


Usaremos o conjunto de dados geográficos (disponível como um arquivo `.shape` com um grafo representando as ruas de São Paulo) distribuído pela plataforma [GeoSampa](https://geosampa.prefeitura.sp.gov.br) da Prefeitura de São Paulo para acessar os endereços disponíveis dentro da distância $d$ da origem.

## WIP

TODOs:
- Unificar .shape de Logradouros e Distritos
- Checar a necessidade de titulos e prep no logradouro (ver comentario)
<!--
print(gdf['lg_titulo'].unique())
Reescrevendo os títulos em extenso 
 gdf['lg_titulo'] = gdf['lg_titulo'].replace({
     'ENG': 'Engenheiro',
     'PROF': 'Professor'
     ''
 })
-->


## Results
Describe and comment the most important results.

## Suggested next steps
State suggested next steps, based on results obtained in this notebook.

### Implementação em Código Python

A implementação segue as etapas principais descritas abaixo:

1. __Carregamento e Preparação dos Dados:__

   - Utilizaremos a biblioteca `geopandas` para manipular arquivos `.shapefile`.
   - 
2. __Cálculo das Probabilidades de Escolha:__

   - Importe os dados do IBGE utilizando `pandas`.
   - Calcularemos a probabilidade de um distrito $\bar{d}$ com base na população e renda dos distritos.
   $$
   \text{Probabilidade}(\bar{d}) = \frac{\text{População}(\bar{d}) \cdot \text{RendaMédia}(\bar{d})}{\sum^{distritos}_{d} (\text{População}(d) \cdot \text{RendaMédia}(d))}
   $$
3. __Seleção do Endereço de Origem:__

   - Escolha aleatória de uma rua com `numpy.random.choice`.
   - Geração de um número dentro do intervalo válido.
4. __Seleção do Endereço de Destino:__

   - Use $P(r)$ para calcular o raio máximo:

     $$
     P(r) = \frac{1}{\sqrt{2\pi}\sigma} e^{-\frac{r^2}{2\sigma^2}}
     $$
   - Escolha uniformemente dentro do raio usando coordenadas do grafo.
5. **Visualização dos Resultados:**

   - Use `matplotlib` ou `folium` para criar mapas interativos.





### Visualização e Validação

Após a implementação, as corridas simuladas podem ser visualizadas em mapas interativos:

- **Mapas com `folium`:** Representam pares de origem e destino.
- **Gráficos:** Analisam estatisticamente a distribuição das distâncias e destinos.

Além disso, as corridas geradas podem ser exportadas para arquivos `.csv` para uso em modelos de treinamento.

# Setup

## Importar bibliotecas
Importamos todas as bibliotecas Python necessárias

In [71]:
# Manipulação de dados
import pandas as pd
import numpy as np


# Opções para exibição de dados
pd.options.display.max_columns = 50
pd.options.display.max_rows = 30

# Visualização de dados
import matplotlib as plt

# Geodata
import geopandas as gpd
from shapely.geometry import Point
from geopy.geocoders import Nominatim
    


# Importação de Dados
Importaremos todos os dados necessários para a análise.

In [72]:
# Carregar os dados do IBGE
df_pop = pd.read_csv('data/population_parameters_rates_by_district.csv')
df_renda = pd.read_csv('data/renda_media_distrito.csv')

# Carregar os dados do geo-dataframe da cidade de São Paulo
gdf = gpd.read_file('/home/sasinhe/uber/data/mapas/SaoPaulo_merged/sao_paulo.shp')



# Processamento de Dados

## Tratamento dso arquivos do IBGE

O resultado dessa seção deve ser um dataframe `df_prob` com os nomes dos distritos e suas respectivas probabilidades

In [73]:
# Remover espaços em branco e converter para minúsculas
df_pop['Nome_distrito'] = df_pop['Nome_distrito'].str.strip().str.lower()
df_renda['Nome_distrito'] = df_renda['Nome_distrito'].str.strip().str.lower()

# Definir intervalos de renda e seus pontos médios
income_intervals = {
    'Menos de 2 SM': 1,
    'De 2 a Menos de 5 SM': 3.5,
    'De 5 a Menos de 10 SM': 7.5,
    'De 10 a Menos de 15 SM': 12.5,
    'De 15 a Menos de 25 SM': 20,
    'De 25 SM e Mais ': 30
}

# Calcular a renda média para cada distrito
for col, midpoint in income_intervals.items():
    df_renda[col] = df_renda[col]/100 * midpoint


df_renda['average_income'] = df_renda[list(income_intervals.keys())].sum(axis=1)

# Normalizar a renda média
min_income = df_renda['average_income'].min()
max_income = df_renda['average_income'].max()
df_renda['normalized_income'] = (df_renda['average_income'] - min_income) / (max_income - min_income)

# Normalizar a população
min_pop = df_pop['Pop_2020'].min()
max_pop = df_pop['Pop_2020'].max()
df_pop['normalized_pop'] = (df_pop['Pop_2020'] - min_pop) / (max_pop - min_pop)

# Mesclar os dois dataframes na coluna 'Nome_distrito'
df_prob = pd.merge(df_pop, df_renda, on='Nome_distrito')

# Calcular a pontuação composta (média simples das normalizações)
df_prob['composite_score'] = (df_prob['normalized_income'] + df_prob['normalized_pop']) / 2

# Normalizar a pontuação composta para obter probabilidades
total_score = df_prob['composite_score'].sum()
df_prob['Probabilidade'] = df_prob['composite_score'] / total_score

# Criar um novo dataframe com os resultados para checagem
# result = df.copy()

# Remover colunas desnecessárias
df_prob = df_prob[["Nome_distrito", "Probabilidade"]]

# Exibir o resultado final com as probabilidades
df_prob

,Nome_distrito,Probabilidade
0,agua rasa,0.009768
1,alto de pinheiros,0.014637
2,anhanguera,0.005868
3,aricanduva,0.007474
4,artur alvim,0.007562
...,...,...
90,vila medeiros,0.008441
91,vila prudente,0.009580
92,vila sonia,0.013279
93,sao domingos,0.008548


## Tratamento de arquivos .shp (mapas)

O resultado final deve ser um geo-dataframe `df_mapa` com as informações de cada logradouro

In [74]:
# Carregar os dados do geo-dataframe da cidade de São Paulo
gdf = gpd.read_file('/home/sasinhe/uber/data/mapas/SaoPaulo_merged/sao_paulo.shp')

gdf.rename(columns={'ds_nomeds_': 'nome_distrito', 'lg_nome': 'nome_logradouro'}, inplace=True) # Renomear colunas
gdf = gdf.loc[gdf['lg_tipo'].isin(['R', 'AV'])] # Filtrar apenas ruas e avenidas
gdf['lg_tipo'] = gdf['lg_tipo'].map({'R' : 'Rua', 'AV': 'Avenida'}) # Mapear códigos para nomes

gdf['num_ini'] = gdf[['lg_ini_par', 'lg_ini_imp']].min(axis=1) # Definir o número inicial
gdf['num_fim'] = gdf[['lg_fim_par', 'lg_fim_imp']].max(axis=1) # Definir o número final
gdf = gdf.dropna(subset=['num_fim']) # Remover valores nulos
gdf = gdf.dropna(subset=['num_ini']) # Remover valores nulos	
gdf[['num_fim', 'num_ini']] = gdf[['num_fim', 'num_ini']].astype(int) # Converter para inteiros
gdf = gdf[gdf['num_ini'] < gdf['num_fim']] # Remover entradas onde num_ini >= num_fim


gdf.drop(columns=['lg_ini_par', 'lg_ini_imp', 'lg_fim_par', 'lg_fim_imp', 'lg_codlog', 'lg_or_geom', 'lg_seg_id', 'lg_id', 'lg_ordem'], inplace=True) # Remover colunas desnecessárias

gdf['logradouro'] = gdf['lg_tipo'] + ' ' + gdf['lg_titulo'].fillna('') + ' ' + gdf['lg_prep'].fillna('') + ' ' + gdf['nome_logradouro']

gdf = gdf.applymap(lambda s: s.lower() if type(s) == str else s) # Deixa todas as entradas que forem string minusculas

/tmp/ipykernel_58762/3216736753.py:20: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  gdf = gdf.applymap(lambda s: s.lower() if type(s) == str else s) # Deixa todas as entradas que forem string minusculas


# Criação do Algoritmo


## Endereço de Origem

In [75]:
def selecionar_distrito_de_origem(prob_distrito_de_origem=df_prob, geodataframe=gdf):
    distritos = prob_distrito_de_origem['Nome_distrito']
    probabilidades = prob_distrito_de_origem['Probabilidade']
    distrito_amostrado = np.random.choice(distritos, p=probabilidades)
    # print(f"Distrito amostrado: {distrito_amostrado}")
    df_ruas = geodataframe[geodataframe['nome_distrito'] == distrito_amostrado]
    return df_ruas

In [76]:
def selecionar_endereço_de_origem():
    df_ruas = selecionar_distrito_de_origem()
    rua_amostrada = df_ruas.sample()
    numero_amostrado = np.random.randint(rua_amostrada['num_ini'].values[0], rua_amostrada['num_fim'].values[0])
    # print(f"Rua amostrada: {rua_amostrada['logradouro'].values[0]}")
    return rua_amostrada, numero_amostrado
    # return str(rua_amostrada['logradouro'].values[0]) + ', ' + str(numero_amostrado) + ', São Paulo SP'

In [77]:
selecionar_endereço_de_origem()[0]

,lg_tipo,lg_titulo,lg_prep,nome_logradouro,nome_distrito,geometry,num_ini,num_fim,logradouro
175727,avenida,None,None,arnolfo azevedo,consolacao,"LINESTRING (329871.11 7395142.192, 329863.708 ...",125,389,avenida arnolfo azevedo


## Endereço de Destino

#### Cálculo da Variável Aleatória $r$
O raio $r$ da distância máxima do endereço de chegada é calculado a partir da distribuição gaussiana com parâmetros $\mu$ e $\sigma$:


$$
P(r) = \frac{1}{\sigma\sqrt{2\pi}} e^{-\frac{(r- \mu)^2}{2\sigma^2}}
$$

Vamos estimar valores razoáveis para $\mu$ e $\sigma$ usando a média e o desvio padrão das distâncias de elementos de `gdf`: 

In [78]:
# TODO Otimizar o cálculo da matriz de distâncias
# import geopandas as gpd
# import numpy as np

# def calcular_distancias_vetorizado(gdf):
#     # Calcular a matriz de distâncias entre todos os pontos
#     dist_matrix = gdf.geometry.apply(lambda geom1: gdf.geometry.distance(geom1))
    
#     # Extrair as distâncias superiores da matriz (evitar duplicatas e zeros)
#     distancias = dist_matrix.values[np.triu_indices_from(dist_matrix, k=1)]
    
#     return distancias

# def estimar_sigma(gdf):
#     distancias = calcular_distancias_vetorizado(gdf)
#     sigma = np.std(distancias)
#     return sigma

# # Estimar sigma
# sigma = estimar_sigma(gdf)

# # Exibir o valor estimado de sigma
# print(f"Sigma estimado: {sigma}")

############################################################################################################################################################################

# No meio tempo, seguem estimativas pros parâmetros baseados em https://www.kaggle.com/code/noohinaaz/uber-driver-complete-data-analysis e na minha cabeça :P

avg = 10 # [Km]
std = 7 # [Km]

### Endereço de Destino

In [79]:
r = np.random.normal(avg, std)  # Gerar um valor aleatório de acordo com a distribuição normal

def selecionar_gdf_de_destino(gdf, rua_origem, distance):
    gdf_destinos = gdf[gdf.geometry.distance(rua_origem.geometry.iloc[0]) <= distance]
    return gdf_destinos

rua_origem = selecionar_endereço_de_origem()[0]
gdf_destinos = selecionar_gdf_de_destino(gdf, rua_origem, r)

In [80]:
def selecionar_endereço_de_destino(gdf, rua_origem, distance):
    rua_amostrada = gdf_destinos.sample()
    # print(rua_amostrada)
    numero_amostrado = np.random.randint(rua_amostrada['num_ini'].values[0], rua_amostrada['num_fim'].values[0])
    # print(f"Rua amostrada: {rua_amostrada['logradouro'].values[0]}")
    return rua_amostrada, numero_amostrado


# Conclusão

Agora, concluiremos o algoritmo criando a função principal

In [81]:


def gerar_corrida(gdf, df_prob, gaussian_avg=10, gaussian_std=7):
    r = abs(np.random.normal(gaussian_avg, gaussian_std))
    print(f"Distância: {r:.2f} Km")
    # Selecionar o endereço de origem
    origem = selecionar_endereço_de_origem()
    rua_origem = origem[0]
    numero_origem = origem[1]
    print(f"Origem: {rua_origem['logradouro'].values[0]}, {numero_origem}")
    
    # Selecionar o endereço de destino
    destino = selecionar_endereço_de_destino(gdf, origem, r)
    rua_destino = destino[0]
    numero_destino = destino[1]
    print(f"Destino: {rua_destino['logradouro'].values[0]}, {numero_destino}")
    
    return [r, rua_origem['logradouro'].values[0], numero_origem, rua_destino['logradouro'].values[0], numero_destino]

test = gerar_corrida(gdf, df_prob)
    
if '__name__' == '__main__':
    gerar_corrida()

Distância: 5.37 Km
Origem: avenida   antonelo da messina, 495
Destino: rua dr  fernando delamain, 152


Vamos gerar uma lista com 10K resultados para as próximas etapas do projeto

In [82]:
df_resultados = pd.DataFrame(columns=['Distância', 'Rua de Origem', 'Número de Origem', 'Rua de Destino', 'Número de Destino'])
for i in range(1000):
    df_resultados.loc[i] = gerar_corrida(gdf, df_prob)
    

df_resultados.to_csv('resultados.csv', index=False)

Distância: 18.70 Km
Origem: avenida   jabaquara, 2091
Destino: rua prof  pinheiro domingues, 627
Distância: 1.04 Km
Origem: rua   elias bauab, 80
Destino: rua dr  fernando delamain, 110
Distância: 4.50 Km
Origem: rua cel  antonio barroso, 293
Destino: rua prof  pinheiro domingues, 529
Distância: 11.21 Km
Origem: avenida dom  pedro i, 272
Destino: rua dr  fernando delamain, 89
Distância: 14.32 Km
Origem: rua   joao daudt filho, 170
Destino: rua dr  jose do amaral, 20
Distância: 1.25 Km
Origem: rua   dene, 10
Destino: rua dr  fernando delamain, 78
Distância: 23.18 Km
Origem: rua   manuel leiroz, 364
Destino: rua dr  jose do amaral, 108
Distância: 10.09 Km
Origem: avenida   pedroso de morais, 1362
Destino: rua dr  jose do amaral, 22
Distância: 2.84 Km
Origem: avenida   jose maria whitaker, 642
Destino: rua dr  fernando delamain, 74
Distância: 8.03 Km
Origem: rua   aniz christofe, 18
Destino: rua dr  jose do amaral, 86
Distância: 2.39 Km
Origem: rua   celestino joaquim da costa, 18
Destino